# Housing assistance

Housing assistance is a payment that varies with income, household size, rental expenses, local economic conditions, and other factors.

### How earnings affect a one-person household's housing assistance

Consider a single person household in San Francisco, California with \$2,000 monthly earned income and \$1,000 monthly rent.
They would be eligible for \$400 per month in housing assistance.


In [1]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px

sim_emp = IndividualSim(year=2022)
sim_emp.add_person(name="person", employment_income=2000 * 12)
sim_emp.add_spm_unit(
    name="spm_unit",
    members=["person"],
    hud_gross_rent=1_000 * 12,
    # 2022 AMI.
    # https://sfmohcd.org/sites/default/files/Documents/MOH/BMR%20Ownership/2022%20AMI-IncomeLimits.pdf
    ami=138550,
    # 2019 1BR payment standard.
    # https://sfmohcd.org/sites/default/files/Documents/MOH/Asset%20Management/2019%20AMI_RentLimits-HMFA.pdf
    pha_payment_standard=2748 * 12,
)

print(
    "Housing assistance: ",
    round(sim_emp.calc("housing_assistance")[0] / 12),
)

HAP:  400


What if their earnings change?
They receive their full rent of \$1,000 if they have zero earnings, and the benefit phases out at 30 cents per dollar of earnings, until it fully phases out at \$3,333 monthly earnings.

In [2]:
sim_emp.vary("employment_income", max=4_000 * 12, step=120)

import plotly.express as px

LABELS = dict(
    employment_income="Monthly employment income",
    dividend_income="Monthly dividend income",
    income="Monthly income",
    income_source="Income source",
    housing_cost="Monthly housing cost",
    housing_assistance="Monthly housing assistance",
    hap_mtr="Marginal tax rate from HAP",
    housing_assistance_mtr="Marginal tax rate from housing assistance",
    hap="Housing assistance payment (if eligible)",
)

emp_df_full = pd.DataFrame(
    dict(
        employment_income=sim_emp.calc("employment_income")[0] / 12,
        hap=sim_emp.calc("hud_hap")[0] / 12,
        housing_assistance=sim_emp.calc("housing_assistance")[0] / 12,
        hap_mtr=-sim_emp.deriv("hud_hap", "employment_income"),
        housing_assistance_mtr=-sim_emp.deriv(
            "housing_assistance", "employment_income"
        ),
    )
)

fig = px.line(
    emp_df_full,
    "employment_income",
    "housing_assistance",
    labels=LABELS,
    title="Housing assistance for a one-person household in California with $1,000 monthly housing costs",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

We can also view their marginal tax rate from the program, revealing housing assistance's 30% phase-out rate.

In [3]:
fig = px.line(
    emp_df_full,
    "employment_income",
    "housing_assistance_mtr",
    labels=LABELS,
    title="Housing assistance MTR for a one-person household in California with $1,000 monthly housing costs",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".0%", yaxis_range=[0, 1]
)
fig.show()